---
Use Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

As you know from the previous assignment, you must tell the Python interpreter where to find Spark before performing any Spark operations in your program.   So, start by doing that:

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

and then create a `SparkContext`:

In [2]:
import random
sc = SparkContext(appName="YourTest", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---

- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [3]:
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    #### Your code for Question 1.1 should go here
    graph = sc.textFile("p2p-Gnutella08-adj.txt")
    nodes = graph.count()
    edges = graph.map(lambda x: x.split("\t")).map(lambda x: len(x)-1).sum()
    return (nodes, edges)

def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the 
    values are the number of nodes of the corresponding outdegree """
    #### Your code for Question 1.2 should go here
    graph = sc.textFile("p2p-Gnutella08-adj.txt")
    edges = graph.map(lambda x: x.split("\t")).map(lambda x: (len(x)-1,1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[0]) 
    return edges.collect()

def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the 
    values are the number of nodes of the corresponding indegree """
    #### Your code for Question 1.3 should go here
    graph = sc.textFile("p2p-Gnutella08-adj.txt")
    edges = graph.map(lambda x: x.split("\t"))\
                 .filter(lambda x: len(x)>1)\
                 .map(lambda x: x[1:])\
                 .flatMap(lambda x: map(lambda y: (y,1), x))\
                 .reduceByKey(lambda x,y: x+y)\
                 .map(lambda x: (x[1],1))\
                 .reduceByKey(lambda x,y: x+y)\
                 .sortBy(lambda x: x[0])
    return edges.collect()




In [4]:
# Test
num_nodes_edges()

(6301, 20777)

In [11]:
# Test
out_counts()

[(0, 3836),
 (1, 294),
 (2, 28),
 (3, 16),
 (4, 28),
 (5, 107),
 (6, 10),
 (7, 9),
 (8, 44),
 (9, 372),
 (10, 1531),
 (11, 1),
 (12, 3),
 (13, 2),
 (14, 2),
 (17, 4),
 (18, 2),
 (19, 1),
 (22, 1),
 (24, 1),
 (25, 1),
 (28, 1),
 (29, 1),
 (31, 1),
 (34, 1),
 (41, 1),
 (46, 1),
 (47, 1),
 (48, 1)]

In [12]:
# Test
in_counts()

[(1, 2452),
 (2, 1287),
 (3, 868),
 (4, 559),
 (5, 333),
 (6, 227),
 (7, 144),
 (8, 76),
 (9, 70),
 (10, 37),
 (11, 29),
 (12, 23),
 (13, 19),
 (14, 13),
 (15, 8),
 (16, 1),
 (18, 2),
 (19, 2),
 (20, 4),
 (21, 2),
 (22, 1),
 (23, 1),
 (25, 1),
 (27, 1),
 (30, 1),
 (31, 2),
 (32, 2),
 (33, 1),
 (35, 1),
 (38, 1),
 (41, 1),
 (44, 1),
 (47, 2),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (54, 1),
 (55, 1),
 (56, 2),
 (57, 1),
 (59, 1),
 (60, 3),
 (61, 1),
 (62, 2),
 (63, 1),
 (66, 2),
 (67, 3),
 (69, 2),
 (70, 3),
 (71, 3),
 (72, 2),
 (73, 2),
 (74, 1),
 (77, 2),
 (78, 1),
 (79, 1),
 (81, 4),
 (82, 1),
 (83, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (91, 1)]

---
Perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.


Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1)

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [5]:
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 2 here
    graph = sc.textFile("p2p-Gnutella08-adj.txt").map(lambda x: x.split("\t"))
    pageRank = graph.map(lambda x: (x[0],0) if x[0] != str(source_node_id) else (x[0],1))
    for i in range(num_iterations):
        # create adjacency matrix
        adjacency_matrix = graph.map(lambda x: (x[0], x[1:])).join(pageRank)\
                                .sortBy(lambda x: x[0])\
                                .map(lambda x: (x[0], 
                                                x[1][0] if len(x[1][0]) != 0 else [str(source_node_id)],
                                                str(source_node_id),
                                                1/len(x[1][0]) if len(x[1][0]) != 0 else 1,
                                                x[1][1]))
        # normal factors
        normalFactors = adjacency_matrix.flatMap(lambda x: map(lambda y: (y, x[3]*(1-jump_factor)*x[4]), x[1])).reduceByKey(lambda x,y: x+y)
        # random jumps
        rands = adjacency_matrix.map(lambda x: (x[2], jump_factor*x[4])).reduceByKey(lambda x,y: x+y)
        # update pagerank
        pageRank = normalFactors.union(rands).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[0])
        
    return pageRank.map(lambda x: (int(x[0]),x[1])).sortBy(lambda x: (x[1], -x[0]), False).take(10)
                            

In [6]:
# Test
personalized_page_rank(0, 5, 0.15)

[(0, 0.36744034928651914),
 (9, 0.03336716175342781),
 (5, 0.03296930694287504),
 (4, 0.03295364825593767),
 (7, 0.03288235094967802),
 (8, 0.03234041924744085),
 (3, 0.03216384352545011),
 (10, 0.03167238811981311),
 (1, 0.03150634323051698),
 (2, 0.03150634323051698)]

---

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

If you were unable to get personalized page rank working in Question 2, replace the code cell below with a text (Markdown) cell, and briefly describe how you *would* have modified your program to incorporate a termination condition based on maximum node change, for partial credit. 

In [7]:
def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 3 here
    graph = sc.textFile("p2p-Gnutella08-adj.txt").map(lambda x: x.split("\t"))
    maxChange = 0.5/graph.count()
    # initial pagerank values
    pageRank = graph.map(lambda x: (x[0],0) if x[0] != str(source_node_id) else (x[0],1))
    
    # adjacency_lists for each node
    adjacency_matrix = graph.map(lambda x: (x[0], (x[1:] if len(x[1:]) != 0 else [str(source_node_id)], 1/len(x[1:]) if len(x[1:]) != 0 else 1)))
    
    # cache adjacency_matrix
    adjacency_matrix.cache()
    while True:
        group = adjacency_matrix.join(pageRank).map(lambda x: (x[0], x[1][0][0], str(source_node_id), x[1][0][1], x[1][1]))
        normalFactors = group.flatMap(lambda x: map(lambda y: (y, x[3]*(1-jump_factor)*x[4]), x[1])).reduceByKey(lambda x,y: x+y)
        rands = group.map(lambda x: (x[2], jump_factor*x[4])).reduceByKey(lambda x,y: x+y)
        pageRankNew = normalFactors.union(rands).reduceByKey(lambda x,y: x+y)
        unions = pageRankNew.union(pageRank).reduceByKey(lambda x,y: x-y if x>= y else y-x).map(lambda x: x[1]).max()
        if unions < maxChange:
            return pageRank.map(lambda x: (int(x[0]),x[1])).sortBy(lambda x: (x[1], -x[0]), False).take(10)
        pageRank = pageRankNew


In [9]:
# Test
import time
start = time.time()
personalized_page_rank_stopping_criterion(0, 0.15)
end = time.time()
print(end-start)

24.581775426864624


In [171]:
personalized_page_rank_stopping_criterion(0, 0.15)

[(0, 0.367454426380708),
 (9, 0.03285649616511095),
 (5, 0.03277630394176993),
 (7, 0.03268689775469701),
 (4, 0.03258036809400745),
 (8, 0.03195681610117242),
 (3, 0.03194432626448388),
 (10, 0.03140811395340886),
 (2, 0.031226254685033407),
 (1, 0.031225627703008508)]

---

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:


I decided to cache the adjacency_matrix structure I created which was created before I start the loop. I had to call upon the adjacency_matrix  in every iteration for mapping each node in the adjacency list with a probability, so by caching it I could reduce recreating it in every loop. This reduced the runtime from 45 seconds to 23 seconds.





---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.